In [1]:
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.keys import Keys
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime as dt
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
import win32com.client as win32
from datetime import datetime
from datetime import date

In [2]:
url = 'http://10.152.157.13:83/api/v3/medidores?pasta=REN871&take=2000'
headers = {'Content-Type': 'application/json','PIM-Auth': 'db3cca0c-1a5f-46e7-98ce-afa5cb3fe4a5'}
req = requests.get(url, headers=headers) 
df = pd.DataFrame(req.json()['items'])

In [3]:
df_ini = pd.read_csv(r"C:\Users\lucas.vieira\Desktop\Acompanhamento Amostrais\efetividade_pim.csv")

In [4]:
datainicio=df_ini.dataHora.max()
today = date.today()
datafim = today.strftime("%Y-%m-%d")

In [5]:
url2 = 'http://10.152.157.13:83/api/v3/execucoes-de-comandos?' + '&execucao-datainicio=' + datainicio +'T00%3A00&execucao-datafim='+ datafim + 'T23%3A59&take=5000000'
headers = {'Content-Type': 'application/json','PIM-Auth': 'db3cca0c-1a5f-46e7-98ce-afa5cb3fe4a5'}
req2 = requests.get(url2, headers=headers)
df_exec = pd.DataFrame(req2.json()['items'])
df_exec['dataHora']=df_exec['dataHora'].str.split('T').str[0]
df2=pd.pivot_table(df_exec, values='id', index=['medidorId','dataHora','status'],  aggfunc={'id': 'count'}).reset_index( )
df2['status']=df2.status.replace({2:"OK",3:"Erro",8:"Parcial",6:"Reagendado",7:"Reagendado",0:"Processando"})
df_pivot = pd.pivot_table(df2, index=['medidorId','dataHora'], columns = 'status',values = 'id',fill_value=0).reset_index()
df=df.drop_duplicates(subset=['medidorId'],keep='last').fillna(0)
lst=df['medidorId']
df_pivot=df_pivot.query('medidorId in @lst')
df_ini=pd.concat([df_ini, pd.merge(df_pivot,df_ini['medidorId'],on='medidorId',how='inner')], axis=0)
df_ini=df_ini.drop_duplicates(subset=['medidorId','dataHora'],keep='last').fillna(0)

In [6]:
df_ini

,medidorId,dataHora,Erro,OK,Parcial,Processando,Reagendado,mes
0,6518,2023-05-01,53,0,0,0.0,42.0,5.0
1,6518,2023-05-02,49,0,0,0.0,46.0,5.0
2,6518,2023-05-03,51,0,0,0.0,44.0,5.0
3,6518,2023-05-04,46,0,0,0.0,49.0,5.0
4,6518,2023-05-05,46,0,0,0.0,49.0,5.0
...,...,...,...,...,...,...,...,...
91339,9489,2023-06-20,47,0,0,0.0,11.0,0.0
91381,9490,2023-06-19,59,0,0,0.0,36.0,0.0
91423,9490,2023-06-20,47,0,0,0.0,11.0,0.0
91465,9491,2023-06-19,6,0,17,0.0,0.0,0.0


In [7]:
df_ini['mes'] = pd.DatetimeIndex(df_ini.dataHora).month

In [8]:
df_ini.to_csv(r"C:\Users\lucas.vieira\Desktop\Acompanhamento Amostrais\efetividade_pim.csv",index = False)

In [9]:
df_nodup=df_ini.drop_duplicates(subset=['medidorId','dataHora'],keep='last').fillna(0)
df_nodup['coletas'] = df_nodup['Parcial'] + df_nodup['OK']
df_nodup['mes'] = pd.DatetimeIndex(df_nodup.dataHora).month
df_nodup['ano'] = pd.DatetimeIndex(df_nodup.dataHora).year
df_nodup

,medidorId,dataHora,Erro,OK,Parcial,Processando,Reagendado,mes,coletas,ano
0,6518,2023-05-01,53,0,0,0.0,42.0,5,0,2023
1,6518,2023-05-02,49,0,0,0.0,46.0,5,0,2023
2,6518,2023-05-03,51,0,0,0.0,44.0,5,0,2023
3,6518,2023-05-04,46,0,0,0.0,49.0,5,0,2023
4,6518,2023-05-05,46,0,0,0.0,49.0,5,0,2023
...,...,...,...,...,...,...,...,...,...,...
91339,9489,2023-06-20,47,0,0,0.0,11.0,6,0,2023
91381,9490,2023-06-19,59,0,0,0.0,36.0,6,0,2023
91423,9490,2023-06-20,47,0,0,0.0,11.0,6,0,2023
91465,9491,2023-06-19,6,0,17,0.0,0.0,6,17,2023


In [10]:
df_instalados = pd.read_excel(r"C:\Users\lucas.vieira\Desktop\Acompanhamento Amostrais\REN871_MED_UC_CHIP_RJ.xlsx", sheet_name='Base')
df_instalados = df_instalados.query("Status == 'Instalado'")
df_date = df_instalados[['Status','Data Instalação']].groupby('Data Instalação').count().reset_index()
df_date['medidores_instalados']=df_date['Status'].cumsum()
df_date = df_date.rename(columns={'Data Instalação': 'dataHora'})
df_date

,dataHora,Status,medidores_instalados
0,2022-08-22,1,1
1,2022-08-25,2,3
2,2022-08-30,17,20
3,2022-08-31,2,22
4,2022-09-01,1,23
...,...,...,...
129,2023-06-13,1,1028
130,2023-06-14,3,1031
131,2023-06-16,2,1033
132,2023-06-17,1,1034


In [11]:
mes=date.today().month
ano=date.today().year

In [12]:
df_efe = df_nodup[['coletas','dataHora','mes','ano']].query("( mes == @mes) and ( ano == @ano) and( coletas >0) ").drop(['ano','mes'],axis=1).groupby(['dataHora']).count().reset_index()
df_efe['dataHora'] = pd.to_datetime(df_efe['dataHora'])
df_efe = df_efe.sort_values(['dataHora'])
tol = pd.Timedelta('10 day')
df_final = pd.merge_asof(df_efe,df_date[['dataHora','medidores_instalados']],on = 'dataHora',direction='nearest',tolerance=tol)
df_final = df_final.ffill()
df_final['efetividade'] = df_final['coletas']/ df_final['medidores_instalados']
df_final['perc_%'] = df_final['efetividade'].map('{:.2%}'.format)
df_final['Dia']=df_final.dataHora.dt.day

In [13]:
efetividade = px.bar(df_final,y='efetividade' ,
                     x = 'Dia',
                     barmode='group', 
                     text = 'perc_%')
efetividade.update_layout(  xaxis=dict(
                            tickmode='linear'
    )
                          
    )
efetividade.update_layout(yaxis_range=[0,1],
                          title_font=dict(size=20))
efetividade.update_layout(xaxis_range=[0,31],
                          title_font=dict(size=20))
efetividade.update_layout(title_text='Efetividade Junho', title_x=0.5)
efetividade.update_traces(textposition="inside", textfont=dict(size=20, color='white'))
efetividade.update_layout(
    autosize=False,
    width=1200,
    height=500)
efetividade.add_shape(type="line", x0=0, y0=0.8, x1=31, y1=0.8, line_width=2, line_color="red")
efetividade.show()
efetividade.write_image("efetividade.png")

In [14]:
df_final= df_final.rename(columns={"dataHora": "dia"})
df_resumo = pd.pivot_table(df_final, columns = df_final.dia.dt.day,values = ['coletas','medidores_instalados','efetividade'])
df_efe=df_resumo.iloc[[1]]
df_efe=df_efe.applymap(lambda x: "{:.2%}".format(x))

In [15]:
df_efe=df_resumo.iloc[[1]]
df_efe=df_efe.applymap(lambda x: "{:.2%}".format(x))
df_coletas = df_resumo.iloc[[0]]
df_coletas=df_coletas.applymap(lambda x: "{:.0f}".format(x))
df_instalados = df_resumo.iloc[[2]]
df_instalados=df_instalados.applymap(lambda x: "{:.0f}".format(x))

In [16]:
df_email=pd.concat([df_instalados,df_coletas,df_efe])

In [17]:
df_email

dia,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
medidores_instalados,1019,1019,1020,1020,1020,1021,1021,1021,1021,1027,1027,1027,1028,1031,1031,1033,1034,1034,1036,1036
coletas,708,707,710,711,713,710,714,708,711,712,710,717,712,720,723,718,717,721,718,715
efetividade,69.48%,69.38%,69.61%,69.71%,69.90%,69.54%,69.93%,69.34%,69.64%,69.33%,69.13%,69.81%,69.26%,69.84%,70.13%,69.51%,69.34%,69.73%,69.31%,69.02%


In [18]:
styler = df_email.style
styler = styler.set_table_styles([{'selector': 'th',
                                   'props': [('background-color', 'blue'),
                                             ('color', 'white'),
                                             ("text-align", "center")]}])
styler.set_caption('Efetividade')

dia,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
medidores_instalados,1019,1019,1020,1020,1020,1021,1021,1021,1021,1027,1027,1027,1028,1031,1031,1033,1034,1034,1036,1036
coletas,708,707,710,711,713,710,714,708,711,712,710,717,712,720,723,718,717,721,718,715
efetividade,69.48%,69.38%,69.61%,69.71%,69.90%,69.54%,69.93%,69.34%,69.64%,69.33%,69.13%,69.81%,69.26%,69.84%,70.13%,69.51%,69.34%,69.73%,69.31%,69.02%


In [19]:
filename = '\efetividade.png'

In [20]:
lista_destino= "lucas.vieira@way2.com.br;thais.leite@way2.com.br;anderson.fagundes@enel.com;waleska.juvenal@way2.com.br;gustavo.bertazzo@enel.com;viviane.nigula@enel.com;weules.correia@enel.com;adriana.paiva@enel.com;marcelo.correa@enel.com;jose.assis@way2.com.br;barbara.goncalves@enel.com;marthely.badika@enel.com;americo.ferreira@enel.com;lindoval.silva@way2.com.br"

In [21]:
#lista_destino= "lucas.vieira@way2.com.br"

In [22]:
# Convert DataFrame to HTML
html_table = df_email.to_html(index=True,classes='center')

# Create Outlook application object
outlook = win32.Dispatch('outlook.application')

# Create a new email message
mail = outlook.CreateItem(0)
mail.To = lista_destino
mail.Subject = ' Informe de Efetividade de Coletas REN871 - RIO DE JANEIRO'
attachment = mail.Attachments.Add( r"C:\Users\lucas.vieira\Desktop\projetos\extracao_efetividade_api" +filename)
attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001E", 'efetividade.png')
# Set the HTML body of the email message
mail.HTMLBody = f'<p>Prezados,<br> Segue relatório de Efetividade de Coletas Diárias do medidores Nansen NSX da REN871.</p>' + styler.to_html()
mail.HTMLBody = mail.HTMLBody + '<br><img src="cid:efetividade.png"><br>'
# Send the email message
mail.Send()